In [2]:
import tensorflow as tf
import numpy as np


In [170]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

In [171]:
x_train = x_train-np.mean(x_train,axis=0)
x_test = x_test - np.mean(x_test,axis=0)

In [172]:
x_train.shape

(50000, 32, 32, 3)

In [161]:
from tensorflow.keras.layers import Dense, Conv2D, GlobalAveragePooling2D, BatchNormalization
input_shape = (32,32,3)
"""
For training on cifar10/cifar100 data
create a plain network model, 6n+2 layers,
"""
def create_model(input_shape,n):
    model = tf.keras.Sequential()
    model.add(Conv2D(filters=16,kernel_size=(3, 3),input_shape=input_shape,activation='relu'
             ,kernel_initializer=tf.keras.initializers.HeNormal(),padding="same",use_bias=True))
    for i in range(2*n):
        model.add(Conv2D(filters=16,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(Conv2D(filters=32,kernel_size=(3, 3),activation='relu',strides=(2,2),padding="same"
             ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    for i in range(2*n-1):
        model.add(Conv2D(filters=32,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(Conv2D(filters=64,kernel_size=(3, 3),activation='relu',strides=(2,2),padding="same"
             ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    for i in range(2*n-1):
        model.add(Conv2D(filters=64,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(GlobalAveragePooling2D())
    model.add(BatchNormalization())
    model.add(Dense(10,activation='softmax'))
    return model

In [85]:
model = create_model(input_shape,3)

In [88]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_260 (Conv2D)          (None, 32, 32, 16)        448       
_________________________________________________________________
conv2d_261 (Conv2D)          (None, 32, 32, 16)        2320      
_________________________________________________________________
conv2d_262 (Conv2D)          (None, 32, 32, 16)        2320      
_________________________________________________________________
conv2d_263 (Conv2D)          (None, 32, 32, 16)        2320      
_________________________________________________________________
conv2d_264 (Conv2D)          (None, 32, 32, 16)        2320      
_________________________________________________________________
conv2d_265 (Conv2D)          (None, 32, 32, 16)        2320      
_________________________________________________________________
conv2d_266 (Conv2D)          (None, 32, 32, 16)      

In [86]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [302]:
# model.fit(x=x_train, 
#           y=y_train,
#           batch_size=128,
#           epochs=10, 
#           validation_data=(x_test, y_test))

In [296]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, GlobalAveragePooling2D, BatchNormalization, ReLU

class ResNet(Model):

    def __init__(self, num_block, input_shape, output_size=100):
        '''
        For training the cifar10/cifar100 data in the ResNet paper
        input_shape: The size of the input. (img_len, img_len, channel_num).
        output_size: The size of the output. It should be equal to the number of classes.
        '''
        super(ResNet, self).__init__()
        #############################################################
        # TODO: Define layers for your custom LeNet network         
        # Hint: Try adding additional convolution and avgpool layers
        #############################################################
        self.num_block = num_block
        self.layer = dict()
        self.layer['conv1'] = Conv2D(filters=16,kernel_size=(3, 3),input_shape=input_shape,activation='relu'
                                ,kernel_initializer=tf.keras.initializers.HeNormal(),padding="same",use_bias=True)

        for i in range(2*num_block):
            self.layer['conv1_%d'%(i+1)] = Conv2D(filters=16,kernel_size=(3, 3),activation='relu',padding="same"
                                    ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())

        self.layer['conv2_1'] = Conv2D(filters=32,kernel_size=(3, 3),activation='relu',strides=(2,2),padding="same"
             ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())
        for i in range(2*num_block-1):
            self.layer['conv2_%d'%(i+2)] = Conv2D(filters=32,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())
        
        self.layer['conv3_1'] = Conv2D(filters=64,kernel_size=(3, 3),activation='relu',strides=(2,2),padding="same"
             ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())
        for i in range(2*num_block-1):
            self.layer['conv3_%d'%(i+2)] = Conv2D(filters=64,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())

        
        self.layer['avgpool'] = GlobalAveragePooling2D()
        self.layer['bn'] = BatchNormalization()
        self.layer['fc'] = Dense(100,activation='softmax')
    
        #############################################################
        #                          END TODO                         #                                              
        #############################################################

    
    def call(self, x):
        '''
        x: input to LeNet model.
        '''
        #call function returns forward pass output of the network
        #############################################################
        # TODO: Implement forward pass for custom network defined 
        # in __init__ and return network output
        #############################################################
        num_block = self.num_block
        x = self.layer['conv1'](x)
        x = BatchNormalization()(x)
        x_prev = x
        
        for i in range(2*num_block):
            if i%2 == 0:
                x = self.layer['conv1_%d'%(i+1)](x)
                x = BatchNormalization()(x)
            else:
                x = ReLU()(self.layer['conv1_%d'%(i+1)](x) + x_prev)
                x = BatchNormalization()(x)
                x_prev = x
        


        for i in range(2*num_block):
            if i%2 == 0:
                x = self.layer['conv2_%d'%(i+1)](x)
                x = BatchNormalization()(x)
            else:
                if x.shape == x_prev.shape:
                    x =  ReLU()(self.layer['conv2_%d'%(i+1)](x) + x_prev)
                    x = BatchNormalization()(x)
                    x_prev = x
                else:
                    x_prev = Conv2D(filters=32,kernel_size=(1, 1),strides=(2, 2)
                            ,use_bias=False,kernel_initializer=tf.keras.initializers.Ones())(x_prev)/16
                    x =  ReLU()(self.layer['conv2_%d'%(i+1)](x) + x_prev)
                    x = BatchNormalization()(x)
                    x_prev = x

        for i in range(2*num_block):
            if i%2 == 0:
                x = self.layer['conv3_%d'%(i+1)](x)
                x = BatchNormalization()(x)
            else:
                if x.shape == x_prev.shape:
                    x =  ReLU()(self.layer['conv3_%d'%(i+1)](x) + x_prev)
                    x = BatchNormalization()(x)
                    x_prev = x
                else:
                    x_prev = Conv2D(filters=64,kernel_size=(1, 1),strides=(2, 2)
                            ,use_bias=False,kernel_initializer=tf.keras.initializers.Ones())(x_prev)/32
                    x =  ReLU()(self.layer['conv3_%d'%(i+1)](x) + x_prev)
                    x = BatchNormalization()(x)
                    x_prev = x

        x = self.layer['avgpool'](x)
        x = self.layer['bn'](x)
        out = self.layer['fc'](x)
        
        return out

In [ ]:
import time
model = ResNet(num_block=1, input_shape=input_shape)


optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

batch_size = 128
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

epochs = 5

history = [[],[],[]]
for epoch in range(epochs):
    print("Epoch %d/%d" % (epoch+1,epochs))
    
    start_time = time.time()

    for step, (x_batch_train, y_batch_train) in enumerate(train_data):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        train_acc_metric.update_state(y_batch_train, logits)
        if step % 400 == 0:
            print(
                "Training loss at step %d: %.4f"
                % (step, float(loss_value))
            )

    history[0].append(loss_value)
    train_acc = train_acc_metric.result()
    train_acc_metric.reset_states()

    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    history[1].append(train_acc)
    history[2].append(val_acc)
    print("Training accuracy: %.4f" % (float(train_acc),)
          ,"Validation accuracy: %.4f" % (float(val_acc),),"Time taken: %.2fs" % (time.time() - start_time))

Epoch 1/5
Training loss at step 0: 5.2383


In [266]:
x = Conv2D(filters=16,kernel_size=(3, 3),input_shape=input_shape,activation='relu'
                                ,kernel_initializer=tf.keras.initializers.HeNormal(),padding="same",use_bias=True)(x)
x_pre = x

In [267]:
x = Conv2D(filters=16,kernel_size=(3, 3),activation='relu',padding="same"
                                    ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())(x)
x = Conv2D(filters=16,kernel_size=(3, 3),activation='relu',padding="same"
                                    ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())(x) + x_pre
x =  ReLU()(x)
x_pre = x

In [268]:
x = Conv2D(filters=32,kernel_size=(3, 3),activation='relu',strides=(2,2),padding="same"
             ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())(x)
x_pre = Conv2D(filters=32,kernel_size=(1, 1),strides=(2, 2)
                            ,use_bias=False,kernel_initializer=tf.keras.initializers.Ones())(x_pre)/16
x = Conv2D(filters=32,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())(x) + x_pre
x =  ReLU()(x)
x_pre = x

In [269]:
x = Conv2D(filters=64,kernel_size=(3, 3),activation='relu',strides=(2,2),padding="same"
             ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())(x)
x_pre = Conv2D(filters=64,kernel_size=(1, 1),strides=(2, 2)
                            ,use_bias=False,kernel_initializer=tf.keras.initializers.Ones())(x_pre)/32
x = Conv2D(filters=64,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal())(x) + x_pre
x =  ReLU()(x)
x_pre = x

In [270]:
x = GlobalAveragePooling2D()(x)

In [271]:
x = Dense(100,activation='softmax')(x)

In [272]:
x

<tf.Tensor: shape=(3, 100), dtype=float32, numpy=
array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        3.2832270e-07, 0.0000000e+00, 0.0000000e+00, 3.9675338e-03,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        9.9603206e-01, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00